### 1) Setear los datos para Los Lagos y aniadir comunas
<p>Las comunas estaran como columnas
<br>0: no hay mencion
<br>1 o mas: se menciona en la noticia (text)</p>

In [114]:
import elasticsearch
from datetime import datetime

In [115]:
country="chile"
from_="2022-01-01"
to_="2022-01-31"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

simple_keyword=False

In [116]:
# https://gist.github.com/juanbrujo/0fd2f4d126b3ce5a95a7dd1f28b3d8dd
comunas= ["Puerto Montt", "Calbuco", "Cochamó", "Fresia", "Frutillar", "Los Muermos", "Llanquihue", "Maullín", "Puerto Varas", "Castro", "Ancud", "Chonchi", "Curaco de Vélez", "Dalcahue", "Puqueldón", "Queilén", "Quellón", "Quemchi", "Quinchao", "Osorno", "Puerto Octay", "Purranque", "Puyehue", "Río Negro", "San Juan de la Costa", "San Pablo", "Chaitén", "Futaleufú", "Hualaihué", "Palena"]


In [117]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [118]:
#NO MODIFICAR

es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

In [119]:
#NO MODIFICAR
query = { 
    "bool": { 
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }
      }},

        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [120]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 1311 noticias encontradas...


In [121]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}

df = pd.DataFrame(data)  
# dfComunas = pd.DataFrame(columns = comunas)

for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    # si text contiene una comuna se agrega a la fila con valor de apariciones
    for comuna in comunas:
        if comuna in text:
            new_row[comuna] = text.count(comuna)
        else:
            new_row[comuna] = 0

    # dfComunas = dfComunas.append(new_row, ignore_index=True)

    df = df.append(new_row, ignore_index=True)

dfComunas = df.drop_duplicates(subset='url', keep='first')

/tmp/ipykernel_9228/4000000255.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_9228/4000000255.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_9228/4000000255.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_9228/4000000255.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_9228/4000000255.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

In [123]:
df = df.drop_duplicates(subset='url', keep='first')
len(df)
df.head()

,id_news,country,media_outlet,url,title,text,date,Ancud,Calbuco,Castro,...,Puqueldón,Purranque,Puyehue,Queilén,Quellón,Quemchi,Quinchao,Río Negro,San Juan de la Costa,San Pablo
0,21907946.0,chile,radiosago,https://www.radiosago.cl/investigan-muerte-de-...,Investigan muerte de hombre apuñalado en Máfil...,La Fiscalía de Los Lagos dirige una investigac...,2022-01-26,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21908053.0,chile,radiosago,https://www.radiosago.cl/ante-retroceso-a-fase...,Ante retroceso a Fase 3: Municipalidad de Puye...,Frente al alza de contagios de Covid-19 en la ...,2022-01-24,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,21908070.0,chile,radiosago,https://www.radiosago.cl/alcaldesa-de-puyehue-...,Alcaldesa de Puyehue dio positivo por covid-19,"La jefa comunal de la comuna lacustre, María J...",2022-01-23,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21908071.0,chile,radiosago,https://www.radiosago.cl/el-primer-semestre-de...,El primer semestre de este año entrará en func...,La jefa de la División de Atención Primaria de...,2022-01-23,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,21908072.0,chile,radiosago,https://www.radiosago.cl/organizaciones-de-la-...,Organizaciones de la región de Los Lagos crean...,Como una necesidad por plantear la urgencia qu...,2022-01-24,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
